## Lab: Neural Networks

## Exercise 1: Regression with Pytorch

In this exercise, we will build a Neural Networks with Pytorch for predicting pollution level. We will be working on the Beijing Pollution dataset:
https://code.datasciencedojo.com/datasciencedojo/datasets/tree/master/Beijing%20PM2.5

The steps are:
1.   Setup Repository
2.   Load and Explore Dataset
3.   Prepare Data
4.   Baseline Model
5.   Define Architecture
6.   Create Data Loader
7.   Train Model
8.   Assess Performance
9.   Push Changes

### 2. Load and Explore Dataset

**[2.0]** Change Working Directory

In [1]:
cd /home/jovyan/work

/home/jovyan/work


In [2]:
! cat /etc/issue*

Ubuntu 20.04.4 LTS \n \l

Ubuntu 20.04.4 LTS


In [5]:
pip install intel_extension_for_pytorch==1.10.100 -f https://software.intel.com/ipex-whl-stable

Looking in links: https://software.intel.com/ipex-whl-stable

× The package index page being used does not have a proper HTML doctype declaration.
╰─> Problematic URL: https://www.intel.com/content/dam/develop/external/us/en/documents/ipex/whl-stable.html

note: This is an issue with the page at the URL mentioned above.
hint: You might need to reach out to the owner of that package index, to get this fixed. See https://github.com/pypa/pip/issues/10825 for context.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 MB 959.6 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


**[2.1]** Download the dataset into the `data/raw` folder:https://code.datasciencedojo.com/datasciencedojo/datasets/raw/master/Beijing%20PM2.5/PRSA_data_2010.1.1-2014.12.31.csv

In [1]:
!wget -P data/raw https://code.datasciencedojo.com/datasciencedojo/datasets/raw/master/Beijing%20PM2.5/PRSA_data_2010.1.1-2014.12.31.csv

--2022-03-07 10:37:45--  https://code.datasciencedojo.com/datasciencedojo/datasets/raw/master/Beijing%20PM2.5/PRSA_data_2010.1.1-2014.12.31.csv
Resolving code.datasciencedojo.com (code.datasciencedojo.com)... 167.99.111.153
Connecting to code.datasciencedojo.com (code.datasciencedojo.com)|167.99.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1966669 (1.9M) [text/plain]
Saving to: ‘../data/raw/PRSA_data_2010.1.1-2014.12.31.csv’

PRSA_data_2010.1.1- 100%[===================>]   1.88M  1.26MB/s    in 1.5s    

2022-03-07 10:37:48 (1.26 MB/s) - ‘../data/raw/PRSA_data_2010.1.1-2014.12.31.csv’ saved [1966669/1966669]



**[2.2]** Launch the magic commands for auto-relaoding external modules

In [3]:
# Task: Launch the magic commands for auto-relaoding external modules
%load_ext autoreload
%autoreload 2

**[2.3]** Import the pandas and numpy packages

In [4]:
import pandas as pd
import numpy as np

**[2.4]** Load the data in a dataframe called `df`

In [6]:
df = pd.read_csv('data/raw/PRSA_data_2010.1.1-2014.12.31.csv')

In [6]:
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [9]:
df.shape

(43824, 13)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 13 columns):
No       43824 non-null int64
year     43824 non-null int64
month    43824 non-null int64
day      43824 non-null int64
hour     43824 non-null int64
pm2.5    41757 non-null float64
DEWP     43824 non-null int64
TEMP     43824 non-null float64
PRES     43824 non-null float64
cbwd     43824 non-null object
Iws      43824 non-null float64
Is       43824 non-null int64
Ir       43824 non-null int64
dtypes: float64(4), int64(8), object(1)
memory usage: 4.3+ MB


In [11]:
df.describe()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,41757.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,21912.500000,2012.000000,6.523549,15.727820,11.500000,98.613215,1.817246,12.448521,1016.447654,23.889140,0.052734,0.194916
std,12651.043435,1.413842,3.448572,8.799425,6.922266,92.050387,14.433440,12.198613,10.268698,50.010635,0.760375,1.415867
min,1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,10956.750000,2011.000000,4.000000,8.000000,5.750000,29.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,21912.500000,2012.000000,7.000000,16.000000,11.500000,72.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,32868.250000,2013.000000,10.000000,23.000000,17.250000,137.000000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,43824.000000,2014.000000,12.000000,31.000000,23.000000,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


### 3. Prepare Data

In [7]:
# Task: Create a copy of df and save it into a variable called df_cleaned
df_cleaned = df.copy()

In [8]:
# Task: Remove the column No as it is an identifier for rows
df_cleaned.drop('No', axis=1, inplace=True)

In [9]:
# Task: Remove the missing values from the target variable pm2.5
df_cleaned.dropna(inplace=True)

In [10]:
# Task: Reset the indexes of the dataframe
df_cleaned.reset_index(drop=True, inplace=True)

In [11]:
# Task: Import StandardScaler and OneHotEncoder from sklearn.preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [12]:
# Task: Create a list called num_cols that contains year, DEWP, TEMP, PRES, Iws, Is, Ir
num_cols = ['year', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']

In [13]:
# Task: Instantiate a StandardScaler and called it sc
sc = StandardScaler()

In [14]:
# Task: Fit and transform the numeric feature of X_train_cleaned and replace the data into it
df_cleaned[num_cols] = sc.fit_transform(df_cleaned[num_cols])

In [15]:
# Task: Create a list called cat_cols that contains Gender
cat_cols = ['month', 'day', 'hour', 'cbwd']

In [16]:
# Task: Instantiate a OneHotEncoder and called it ohe
ohe = OneHotEncoder(sparse=False)

In [17]:
# Task: Perform One-Hot encoding on cat_cols and save them into a dataframe called X_cat
X_cat = pd.DataFrame(ohe.fit_transform(df_cleaned[cat_cols]))

In [18]:
# Task: Extract the feature names from ohe and replace the names of the columns of the X_cat
X_cat.columns = ohe.get_feature_names(cat_cols)

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
# Task: Drop the original columns of cat_cols from df_cleaned
df_cleaned.drop(cat_cols, axis=1, inplace=True)

In [20]:
# Task: Concatenate df_cleaned with X_cat and save the result to a variable called X
X = pd.concat([df_cleaned, X_cat], axis=1)

In [21]:
cd /home/jovyan/work

/home/jovyan/work


In [22]:
# Task: Import train_test_split from sklearn.model_selection
from src.data.sets import split_sets_by_time, save_sets

In [23]:
# Task: Split the data into training and testing sets with 80-20 ratio
X_train, y_train, X_val, y_val, X_test, y_test = split_sets_by_time(X, target_col='pm2.5', test_ratio=0.2)

In [23]:
!mkdir data/processed/beijing_pollution

mkdir: cannot create directory ‘data/processed/beijing_pollution’: File exists


In [24]:
# Task: Save the sets in the data/processed/beijing_pollution folder
save_sets(X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, path='data/processed/beijing_pollution/')

### 4. Baseline Model

In [25]:
# Task: Import NullModel from src.models.null
from src.models.null import NullModel

In [26]:
# Task: Instantiate a NullModel and call .fit_predict() on the training target to extract your predictions into a variable called y_base
base_model = NullModel(target_type="regression")
y_base = base_model.fit_predict(y_train)

In [27]:
from src.models.performance import print_reg_perf

In [28]:
print_reg_perf(y_base, y_train, set_name='Base')

RMSE Base: 92.82545840756482
MAE Base: 69.67082209440568


### 5. Define Architecture

In [29]:
# Task: Import torch and torch.nn as nn
import torch
import torch.nn as nn
import torch.nn.functional as F

**[5.2]** Create in `src/models/pytorch.py` a class called `PytorchRegression` that inherits from `nn.Module` with:
- `num_features` as input parameter
- attributes:
    - `layer_1`: fully-connected layer with 128 neurons
    - `layer_out`: fully-connected layer with 1 neurons
- methods:
    - `forward()` with `inputs` as input parameter, perform ReLU and DropOut on the fully-connected layer followed by the output layer

In [30]:
# Task: Create a class called PytorchRegression that inherits from nn.Module
class PytorchRegression(nn.Module):
    def __init__(self, num_features):
        super(PytorchRegression, self).__init__()
        
        self.layer_1 = nn.Linear(num_features, 128)
        self.layer_out = nn.Linear(128, 1)

    def forward(self, x):
        x = F.dropout(F.relu(self.layer_1(x)))
        x = self.layer_out(x)
        return (x)

In [36]:
# Task: Instantiate PytorchRegression with the correct number of input feature and save it into a variable called model
# from src.models.pytorch import PytorchRegression

In [31]:
model = PytorchRegression(X_train.shape[1])

In [32]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

In [33]:
device = get_device()
model.to(device)

PytorchRegression(
  (layer_1): Linear(in_features=78, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=1, bias=True)
)

In [34]:
# Task: Print the architecture of model
print(model)

PytorchRegression(
  (layer_1): Linear(in_features=78, out_features=128, bias=True)
  (layer_out): Linear(in_features=128, out_features=1, bias=True)
)


In [35]:
torch.cuda.is_available()

False

### 6. Create Data Loader

In [36]:
# Task: Import Dataset and DataLoader from torch.utils.data
from torch.utils.data import Dataset, DataLoader

In [38]:
# Task: Create a class called PytorchDataset
class PytorchDataset(Dataset):
    """
    Pytorch dataset
    ...

    Attributes
    ----------
    X_tensor : Pytorch tensor
        Features tensor
    y_tensor : Pytorch tensor
        Target tensor

    Methods
    -------
    __getitem__(index)
        Return features and target for a given index
    __len__
        Return the number of observations
    to_tensor(data)
        Convert Pandas Series to Pytorch tensor
    """
        
    def __init__(self, X, y):
        self.X_tensor = self.to_tensor(X)
        self.y_tensor = self.to_tensor(y)
    
    def __getitem__(self, index):
        return self.X_tensor[index], self.y_tensor[index]
        
    def __len__ (self):
        return len(self.X_tensor)
    
    def to_tensor(self, data):
        return torch.Tensor(np.array(data))

In [42]:
# Task: Convert all numpy array sets to PytorchDataset
train_dataset = PytorchDataset(X=X_train, y=y_train)
val_dataset = PytorchDataset(X=X_val, y=y_val)
test_dataset = PytorchDataset(X=X_test, y=y_test)

### 7. Train Model

In [43]:
# Task: Instantiate a nn.MSELoss() and save it into a variable called criterion
criterion = nn.MSELoss()

In [44]:
# Task: Instantiate a torch.optim.Adam() optimizer with the model's parameters and 0.001 as learning rate and save it into a variable called optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [45]:
# Create a function called `train_regression()` that will perform forward and back propagation and 
# calculate loss and RMSE scores

def train_regression(train_data, model, criterion, optimizer, batch_size, device, scheduler=None, collate_fn=None):
    """Train a Pytorch regresssion model

    Parameters
    ----------
    train_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    optimizer: torch.optim
        Optimizer
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    scheduler : torch.optim.lr_scheduler
        Pytorch Scheduler used for updating learning rate
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        RMSE Score
    """
    
    # Set model to training mode
    model.train()
    train_loss = 0

    # Create data loader
    data = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Reset gradients
        optimizer.zero_grad()
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Make predictions
        output = model(feature)
        
        # Calculate loss for given batch
        loss = criterion(output, target_class)
        
        # Calculate global loss
        train_loss += loss.item()
        
        # Calculate gradients
        loss.backward()
        
        # Update Weights
        optimizer.step()
        
    # Adjust the learning rate
    if scheduler:
        scheduler.step()

    return train_loss / len(train_data), np.sqrt(train_loss / len(train_data))

In [46]:
# Task: Create a function called `test_regression()` that will perform forward and calculate loss and RMSE scores

def test_regression(test_data, model, criterion, batch_size, device, collate_fn=None):
    """Calculate performance of a Pytorch regresssion model

    Parameters
    ----------
    test_data : torch.utils.data.Dataset
        Pytorch dataset
    model: torch.nn.Module
        Pytorch Model
    criterion: function
        Loss function
    bacth_size : int
        Number of observations per batch
    device : str
        Name of the device used for the model
    collate_fn : function
        Function defining required pre-processing steps

    Returns
    -------
    Float
        Loss score
    Float:
        RMSE Score
    """    
    
    # Set model to evaluation mode
    model.eval()
    test_loss = 0

    # Create data loader
    data = DataLoader(test_data, batch_size=batch_size, collate_fn=collate_fn)
    
    # Iterate through data by batch of observations
    for feature, target_class in data:
        
        # Load data to specified device
        feature, target_class = feature.to(device), target_class.to(device)
        
        # Set no update to gradients
        with torch.no_grad():
            
            # Make predictions
            output = model(feature)
            
            # Calculate loss for given batch
            loss = criterion(output, target_class)
            
            # Calculate global loss
            test_loss += loss.item()
            
    return test_loss / len(test_data), np.sqrt(test_loss / len(test_data))


In [48]:
# Task: Create 2 variables called N_EPOCHS and BATCH_SIZE that will take respectively 5 and 32 as values
N_EPOCHS = 5
BATCH_SIZE = 32

In [52]:
# Task: Create a for loop that will iterate through the specified number of epochs and 
# will train the model with the training set and assess the performance on the validation set and print their scores

for epoch in range(N_EPOCHS):
    train_loss, train_rmse = train_regression(train_dataset, model=model, criterion=criterion, optimizer=optimizer, batch_size=BATCH_SIZE, device=device)
    valid_loss, valid_rmse = test_regression(val_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)

    print(f'Epoch: {epoch}')
    print(f'\t(train)\tLoss: {train_loss:.4f}\t|\tRMSE: {train_rmse:.1f}')
    print(f'\t(valid)\tLoss: {valid_loss:.4f}\t|\tRMSE: {valid_rmse:.1f}')

    

Epoch: 0
	(train)	Loss: 272.1585	|	RMSE: 16.5
	(valid)	Loss: 236.9672	|	RMSE: 15.4
Epoch: 1
	(train)	Loss: 272.1389	|	RMSE: 16.5
	(valid)	Loss: 237.1508	|	RMSE: 15.4
Epoch: 2
	(train)	Loss: 272.0000	|	RMSE: 16.5
	(valid)	Loss: 237.7895	|	RMSE: 15.4
Epoch: 3
	(train)	Loss: 272.2039	|	RMSE: 16.5
	(valid)	Loss: 237.1903	|	RMSE: 15.4
Epoch: 4
	(train)	Loss: 271.8850	|	RMSE: 16.5
	(valid)	Loss: 237.0916	|	RMSE: 15.4


In [50]:
# Task: Save the model into the models folder
torch.save(model, "models/pytorch_reg_pm2_5.pt")

### 8.   Assess Performance

In [51]:
# Task: Assess the model performance on the testing set and print its scores
test_loss, test_rmse = test_regression(test_dataset, model=model, criterion=criterion, batch_size=BATCH_SIZE, device=device)
print(f'\tLoss: {test_loss:.4f}\t|\tRMSE: {test_rmse:.1f}')

	Loss: 276.4204	|	RMSE: 16.6


In [56]:
# Task: Add Changes to GIT
! git add .
! git commit -m "pytorch regression"

[pytorch_reg 03b60a8] pytorch regression
 23 files changed, 4386 insertions(+), 1004 deletions(-)
 create mode 100644 models/pytorch_reg_pm2_5.pt
 create mode 100644 notebooks/1_pytorch_regression.ipynb
 create mode 100644 notebooks/Group1-Assignment1-LogisticRegression.ipynb
 create mode 100644 src/data/sets.py
 create mode 100644 src/models/null.py
 create mode 100644 src/models/performance.py
 create mode 100644 src/models/pytorch.py


In [55]:
! git config --global user.email "nathan@fragar.id.au"
! git config --global user.name "Nathan Fragar"

In [59]:
! git push --set upstream origin pytorch_reg

error: src refspec origin does not match any
error: failed to push some refs to 'upstream'
